# 🚀 Working with Objects & Domo Integration
In this tutorial we will:
1. store authentication credentials securely using Domo Account Objects
2. learn to interact with Domo Account objects in Jupyter
3. implement a workflow that interrogates Domo APIs using session_token authentication
4. output the results to a dataset


## 🎓 Before we begin:
- notebooks allow you to compartmentalize your code, write a simple test under each function
- storing functios in .py file allow you to import files as modules and recycle code in the same workspace
    - you may need a blank file called `__init__.py` in all the subfolders

### ▶️ store `get_session_token()` in a file `functions/auth.py`
    
- ensure ./functions has  a blank file called `__init__.py`
- try importing your function like a 'normal' module

### ▶️ store `get_accounts()` in a file `accounts.py`


## 🚀 Handle sensitive credentials appropriately

Do not store sensitive credentials in plain text on the internet!<br>

### 🧪 Use [python-dotenv](https://pypi.org/project/python-dotenv/) to store a local `.env` and then make sure to `.gitignore` it

⚠️ DomoJupyterWorkspaces makes it difficult to see the .env file so you can use env.txt  Be aware that anyone with access to your notebook can see the .txt file.

In [1]:
# %pip install python-dotenv

In [6]:
from dotenv import load_dotenv
import os

load_dotenv('env.txt', override= True)

os.environ['DOMO_PASSWORD']

'thisissecure'

### 🧪 Use Domo Account objects to store credentials
- secured fields can only be seen in clear text in DomoJupyter and use the same encryption platform as Domo connectors.
- ⚠️if you have accounts-v2 (account sharing) enabled in your instance, any user with whom you've shared the connector can read your credentials

### ▶️ create account an "abstract_credentials_store" and "access_token" account object with your username and password (Data > Accounts).
1. create an Abstract_Credentials_Store_Account named `YourInitials_Absract`
   - store your credentials as a properly formatted json object
   ```
   {
      "DOMO_USERNAME": "<your_username>", 
      "DOMO_PASSWORD": "<your_password>",
      "DOMO_INSTANCE": "<domo_instance>"
   }
   ```
2. create a Domo_Access_Token_Account named `YourInitials_AccessToken`
3. edit this workspace and share the accounts with this notebook


### ▶️ create an ouput dataset for your JupyterWorkspace called `YourInitials_MONIT_DomoAccount`.


## 🚀 Read Account Objects in Jupyter Workspaces

### ▶️ write a function `read_domo_jupyter_account` that reads in the properties of an account 

- Implement function in `functions/utils.py`
- function should receive one argument `account_name` and return a dictionary representing the account properties

Notice that you can see secure fields in plain text.  

```
 {
    "prop1": "value",
    "prop2" : "value"
 }
 ```

In [3]:
# from solutions.read_domo_jupyter_account_v1 import read_domo_jupyter_account
import domojupyter as dj

# FIX ME and move to functions/utils.py
def read_domo_jupyter_account(account_name):
    # account_properties = dj.get_account_property_keys('ACCOUNT_NAME')
    # account_property_value = dj.get_account_property_value('ACCOUNT_NAME', account_properties[0])
    # creds = { fix me!! }
    return creds

ModuleNotFoundError: No module named 'domojupyter'

In [ ]:
# from solutions.read_domo_jupyter_account_v1 import read_domo_jupyter_account

ACCOUNT_NAME = "your_account_name" 
creds = read_domo_jupyter_account(ACCOUNT_NAME)

NameError: name 'read_domo_jupyter_account' is not defined

### 🧪 modify `read_domo_jupyter_account` to provide custom formatting to the output for `abstract_credentials_store` account types

🎓 you could have two functions, one for handling any account type, and one specifically for formatting abstract_credentials_store accounts.  But no need to overengineer today :D

1. Modify `read_domo_jupyter_account` to receive parameter, `is_abstract : bool = False`
2. add a conditional return to return `if not is_abstract_account`: return creds` 
3. use `json.load()` to convert the `creds['credentials']` from a string into a dictioary  
4. return the result

[solution](./solutions/read_domo_jupyter_account_v2.py)

## ▶️ Put it all Together with a function called 'main'

create a function called `main()` that encapsulates the entire script.

Main should:
1. retrieve username and password from domo account, `account_name` object using `read_domo_jupyter_account`
2. get a session token using `get_session_token`
3. authenticate our API request to `get_accounts`


In [ ]:
from typing import List

# from solutions.read_domo_jupyter_account_v2 import read_domo_jupyter_account
# from solutions.get_session_token import get_session_token
# from solutions.get_accounts_v2 import get_accounts

def main(account_name) -> List[dict]:
    pass



In [ ]:
account_ls = main(ACCOUNT_NAME)

## 🎓 What's the deal with main?

All of our code so far consists of individual functions are are each test-able on their own.

This vastly improves code legibility and speaks to decoupling (making sure peices of code can stand on their own because they just perform one task with a very short list of inputs).

Main becomes an 'implementation' or a 'program' that we might repeat multiple times, and the component parts (the .py files) are peices we are recycling across multiple implementations

## ▶️ Output to Dataset

1. use pandas to convert our list of results into a dataframe.
2. write the dataframe to the dataset `YourInitials_MONIT_DomoAccount`

In [ ]:
# %pip install pandas

In [ ]:
import pandas as pd
import domojupyter as dj

df = pd.DataFrame(account_ls)
dj.write_dataframe(df, "YourInitials_MONIT_DomoAccount")

## 🚀 Solution

In [14]:
from solutions.utils import read_domo_jupyter_account
from solutions.auth import get_session_token
from solutions.accounts import get_accounts

def main(account_name) -> List[dict]:
    creds = read_domo_jupyter_account(account_name)

    domo_username = creds.get('DOMO_USERNAME')
    domo_password = creds.get('DOMO_PASSWORD')
    domo_instance = creds.get('DOMO_INSTANCE')

    session_token = get_session_token(domo_username=domo_username, domo_password=domo_password, )
    res = get_accounts(domo_instance = domo_instance , session_token= session_token)

    account_ls = res.response

    df = pd.DataFrame(account_ls)
    dj.write_dataframe(df, "YourInitials_MONIT_DomoAccount")

main()
                              